In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/churn-modelling/Churn_Modelling.csv


# Credits and Links

* UDEMY - Machine Learning A-Z <sup>TM</sup>*

<a id="index"></a>
# Table of Content

1. [Importing Libraries](#1)
2. [Data Preprocessing](#2)
- 2.1 [Importing Dataset](#2.1)
- 2.2 [Encoding Categorical Data](#2.2)
- 2.3 [Splitting Dataset into Training and Test set](#2.3)
- 2.4 [Feature Scaling](#2.4)
3. [Building the ANN (Artificial Neural Network)](#3)
- 3.1 [Initializing ANN](#3.1)
- 3.2 [Adding input layer + 1st hidden layer](#3.2)
- 3.3 [Adding 2nd hidden layer](#3.3)
- 3.4 [Adding the output layer](#3.4)
4. [Training the ANN](#4)
- 4.1 [Compiling the ANN](#4.1)
- 4.2 [Training the ANN on the Training set](#4.2)
5. [Predictions and Model Evaluation](#5)
- 5.1 [Predicting result of single observation](#5.1)
- 5.2 [Predicting Test set results](#5.2)
- 5.3 [Confusion Matrix](#5.3)

<a id="1"></a>
# 1. Importing Libraries

[Go back to Index](#index)

In [2]:
#1. General
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

#2. Dataprecossing
#2.2 Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

#2.3 Train test split
from sklearn.model_selection import train_test_split

#2.4 Feature Scaling
from sklearn.preprocessing import StandardScaler

#3. ANN Library
import tensorflow as tf

#5. Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score

In [3]:
tf.__version__

'2.13.0'

<a id="2"></a>
# 2. Data Preprocessing

[Go back to Index](#index)

<a id="2.1"></a>
## 2.1 Importing the dataset

[Go back to Index](#index)

In [4]:
df = pd.read_csv("/kaggle/input/churn-modelling/Churn_Modelling.csv")

In [5]:
df.head(2)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [6]:
X = df.iloc[:, 3:-1].values #Start with the column CreditScore (index=3) as the previous columns are irrelevant 
y = df.iloc[:, -1].values

In [7]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [8]:
y

array([1, 0, 1, ..., 1, 1, 0])

<a id="2.2"></a>
## 2.2 Encoding Categorical Data

[Go back to Index](#index)

### 2.2.1 Label Encoding the `Gender` column 

In [9]:
le = LabelEncoder()

In [10]:
X[:5, 2]

array(['Female', 'Female', 'Female', 'Female', 'Female'], dtype=object)

In [11]:
#Gender Column
X[:, 2] = le.fit_transform(X[:, 2]) 

In [12]:
#Label Encoder randomly assigns values of 1/0 for the Gender column, please do not associate anything with regards to 0/1
X[:5, 2]

array([0, 0, 0, 0, 0], dtype=object)

### 2.2.2 OneHotEncoding the `Geography` column 

In [13]:
X[:5, 1]

array(['France', 'Spain', 'France', 'France', 'Spain'], dtype=object)

In [14]:
ct = ColumnTransformer(
    transformers=[(
            "encoder",
            OneHotEncoder(),
            [1] #index number for the Geography column
    )],
    remainder = "passthrough"
)

In [15]:
X = np.array(ct.fit_transform(X))

In [16]:
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

<a id="2.3"></a>
## 2.3 Splitting Dataset into Training and Test set

[Go back to Index](#index)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

<a id="2.4"></a>
## 2.4 Feature Scaling

[Go back to Index](#index)

In [18]:
sc = StandardScaler()

In [19]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

<a id="3"></a>
# 3. Building the ANN (Artificial Neural Network) Model

[Go back to Index](#index)

<a id="3.1"></a>
## 3.1 Initializing ANN

[Go back to Index](#index)

In [20]:
ann = tf.keras.models.Sequential()

<a id="3.2"></a>
## 3.2 Adding input layer + 1st Hidden layer

[Go back to Index](#index)

<a href="https://www.kaggle.com/code/dansbecker/rectified-linear-units-relu-in-deep-learning"> ReLu (Rectified Linear Unit - Activation Function</a>

<a href="https://deepai.org/machine-learning-glossary-and-terms/sigmoid-function">Sigmoid - Activation Function</a>

In [21]:
#units:= # of input columns
ann.add(tf.keras.layers.Dense(units=6, activation="relu"))

<a id="3.3"></a>
## 3.3 Adding 2nd Hidden layer

[Go back to Index](#index)

In [22]:
#Same LOC as above, 6 hidden nodes
ann.add(tf.keras.layers.Dense(units=6, activation="relu"))

<a id="3.4"></a>
## 3.4 Adding Output Layer

[Go back to Index](#index)

In [23]:
#Only 1 Dependent Column so units=1
#Sigmoid activation function to be applied on output layer
ann.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

<a id="4"></a>
# 4. Training the ANN

[Go back to Index](#index)

<a href="https://www.simplilearn.com/tutorials/machine-learning-tutorial/what-is-epoch-in-machine-learning">Epochs in Neural Network</a>

<a id="4.1"></a>
## 4.1 Compiling the ANN

[Go back to Index](#index)

In [24]:
ann.compile(
    optimizer = "Adam", #best for stochastic gradient descent
    loss = "binary_crossentropy", #for binary classification
    metrics = ["accuracy"]
)

<a id="4.2"></a>
## 4.2 Training the ANN on the Training Set

[Go back to Index](#index)

In [25]:
#Default/Classic Batch Size = 32 for stochastic gradient
#epochs:= number of passes for training
ann.fit(X_train, y_train, batch_size = 32, epochs=100)

Epoch 1/100
250/250 [==============================] - 2s 2ms/step - loss: 0.6783 - accuracy: 0.5789
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4748 - accuracy: 0.7981
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4424 - accuracy: 0.8029
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4344 - accuracy: 0.8021
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4290 - accuracy: 0.8025
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4239 - accuracy: 0.8040
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4178 - accuracy: 0.8058
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4085 - accuracy: 0.8096
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3973 - accuracy: 0.8136
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3846 - accura

<a id="5"></a>
# 5. Predictions and Evaluating the Model

[Go back to Index](#index)

<a id="5.1"></a>
## 5.1 Predicting the result of a single observation

[Go back to Index](#index)

### Scenario1

Use the ANN model to predict if the customer with the following information will leave the bank:

1. Geography:= France
2. Credit Score:= 600
3. Gender:= Male
4. Age:= 40 years old
5. Tenure:= 3 Years
6. Balance:= 60,000 USD
7. Number of Products:= 2
8. Has a Credit Card?:= Yes
9. Active Member?:= Yes
10. Estimated Salary:= 50,000 USD

In [26]:
ann.predict(
    sc.transform([[
    1, 0, 0, #One Hot Encoding Dummy Variables for Geography - France
    600,
    1, #One Hot Encoding Variable for Gender - Male
    40, 3, 60000, 2, 1, 1, 50000
]]))

1/1 [==============================] - 0s 136ms/step


array([[0.02938843]], dtype=float32)

### Conclusion

This customer has a `very low` chance of leaving the bank.

<a id="5.2"></a>
## 5.2 Predicting the result of Test set

[Go back to Index](#index)

In [27]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5) #binary classification prediction

63/63 [==============================] - 0s 1ms/step


In [28]:
np.concatenate((
        y_pred.reshape(len(y_pred), 1),
        y_pred.reshape(len(y_test), 1)
    )
,1)

array([[False, False],
       [False, False],
       [False, False],
       ...,
       [False, False],
       [False, False],
       [False, False]])

<a id="5.3"></a>
## 5.3 Confusion Matrix

[Go back to Index](#index)

In [29]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[1516,   79],
       [ 197,  208]])

* TP:= 1520 Correct predictions that Customer stays in the bank
* TN:= 200 Correct predictions that Customer leaves the bank
* FP:= 75 incorrect predictions that Customer stays in the bank
* FN:= 205 inorrect predictions that Customer leaves the bank

In [30]:
accuracy_score(y_test, y_pred)

0.862